# Core Concepts

## Cluster Architecture

### MasterNodes
- API Server
- etcd
- Scheduler
- Controller Manager

### Worker Node
- Kubelet
- Container Runtime Engine
- Kube-proxy

---

## Etcd

distributed key-value store, simple, reliable, fast

1. download binary
1. extract
1. run etcd
   ```
   ./etcd
   ```
   etcd listens on port **2379** by default
   
   default cli client `etcdctl`
   ```
   ./etcdctl set <ke1> <valu1>
   ./etcdctl get <ke1>|
   ```
---

## Etcd in K8s


### k8s setup

1. Manual
   etcd is run as a binary in master node, etcd listens on maspternodeip, port 2379
2. kubdeadm
   in a cluster set ups using kubeadm, etcd is run as a pod in kubesystem

In [5]:
kubectl get pods -n kube-system

NAME                                        READY   STATUS    RESTARTS   AGE
coredns-5c98db65d4-6n8rw                    1/1     Running   4          6d1h
coredns-5c98db65d4-wt29k                    1/1     Running   4          6d1h
etcd-minikube                               1/1     Running   2          6d1h
kube-addon-manager-minikube                 1/1     Running   2          6d1h
kube-apiserver-minikube                     1/1     Running   2          6d1h
kube-controller-manager-minikube            1/1     Running   2          6d1h
kube-proxy-cdsjx                            1/1     Running   2          6d1h
kube-scheduler-minikube                     1/1     Running   2          6d1h
kubernetes-dashboard-7b8ddcb5d6-strk2       1/1     Running   5          6d1h
nginx-ingress-controller-5d9cf9c69f-jv79n   1/1     Running   2          6d1h
storage-provisioner                         1/1     Running   5          6d1h


In [21]:
kubectl exec etcd-minikube -n kube-system  etcdctl get / --prefix -keys-only #fix this later

Error: unknown flag: --prefix


Examples:
  # Get output from running 'date' command from pod mypod, using the first container by default
  kubectl exec mypod date
  
  # Get output from running 'date' command in ruby-container from pod mypod
  kubectl exec mypod -c ruby-container date
  
  # Switch to raw terminal mode, sends stdin to 'bash' in ruby-container from pod mypod
  # and sends stdout/stderr from 'bash' back to the client
  kubectl exec mypod -c ruby-container -i -t -- bash -il
  
  # List contents of /usr from the first container of pod mypod and sort by modification time.
  # If the command you want to execute in the pod has any flags in common (e.g. -i),
  # you must use two dashes (--) to separate your command's flags/arguments.
  # Also note, do not surround your command and its flags/arguments with quotes
  # unless that is how you would execute it normally (i.e., do ls -t /usr, not "ls -t /usr").
  kubectl exec mypod -i -t -- ls -t /usr
  
  # Get output from running 

: 1

---

## Kube-API server

- Authenticate
- Validate
- Retrieve data from Etcd
- Update Etcd
- Schduler (watches api-server and responds to changes)
- Kubelet (use api-server to perform actions)

in kubeadm setup apiserver params can be viewed from /etc/kubenetes/manifests


```
minikube ssh
cat /etc/kubernetes/manifests

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    component: kube-apiserver
    tier: control-plane
  name: kube-apiserver
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-apiserver
    - --advertise-address=192.168.39.11
    - --allow-privileged=true
    - --authorization-mode=Node,RBAC
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --enable-admission-plugins=NamespaceLifecycle,LimitRanger,ServiceAccount,DefaultStorageClass,DefaultTolerationSeconds,NodeRestriction,Mutati
ngAdmissionWebhook,ValidatingAdmissionWebhook,ResourceQuota
    - --enable-bootstrap-token-auth=true
    - --etcd-cafile=/var/lib/minikube/certs/etcd/ca.crt
    - --etcd-certfile=/var/lib/minikube/certs/apiserver-etcd-client.crt
    - --etcd-keyfile=/var/lib/minikube/certs/apiserver-etcd-client.key
    - --etcd-servers=https://127.0.0.1:2379
    - --insecure-port=0
...
```

on master node we can inspect `kube-apiserver` process
```
ps -aux | grep kube-apiserver
```
---

## Kube Controller Manager

controllers watch a system component and recociles component state

controller binary start a controller manager
`--controller` option 

### examples

1. Node Controller
- watches node heart beats every 5s (--node-monitor-period)
- if there is no heartbeat, the node is marked as unreachable after 40s (--node-monitor-grace-period)
- wati for 5mins for the node to comeback, if node doesn't comeback rechedule pods (--pod-eviction-timeout)

2. Replication Controller

In kubeadm setup, controller manager runs as a pod

```
minikube ssh
cat /etc/kubernetes/manifests/kube-controller-manager.yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    component: kube-controller-manager
    tier: control-plane
  name: kube-controller-manager
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-controller-manager
    - --authentication-kubeconfig=/etc/kubernetes/controller-manager.conf
    - --authorization-kubeconfig=/etc/kubernetes/controller-manager.conf
    - --bind-address=127.0.0.1
    - --client-ca-file=/var/lib/minikube/certs/ca.crt
    - --cluster-signing-cert-file=/var/lib/minikube/certs/ca.crt
    - --cluster-signing-key-file=/var/lib/minikube/certs/ca.key
    - --controllers=*,bootstrapsigner,tokencleaner
    - --kubeconfig=/etc/kubernetes/controller-manager.conf
    - --leader-elect=true
    - --requestheader-client-ca-file=/var/lib/minikube/certs/front-proxy-ca.crt
    - --root-ca-file=/var/lib/minikube/certs/ca.crt
    - --service-account-private-key-file=/var/lib/minikube/certs/sa.key
    - --use-service-account-credentials=true
    image: k8s.gcr.io/kube-controller-manager:v1.16.0
    imagePullPolicy: IfNotPresent
...
```

on master node we can inspect kube-apiserver process

ps -aux | grep kube-controller-manager

---

## Kube Scheduler

decides which pods goes on which node

scheduler goes through 2 phases

1. filter nodes
2. rank nodes to find best fit (assign score 0-10)

schduler binary can be downloaded from kubernetes release page and run

if the cluster is setup using kubeadm, schedulre is setup as a pod

```
minikube ssh
cat /etc/kubernetes/manifests/kube-controller-manager.yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: null
  labels:
    component: kube-scheduler
    tier: control-plane
  name: kube-scheduler
  namespace: kube-system
spec:
  containers:
  - command:
    - kube-scheduler
    - --authentication-kubeconfig=/etc/kubernetes/scheduler.conf
    - --authorization-kubeconfig=/etc/kubernetes/scheduler.conf
    - --bind-address=127.0.0.1
    - --kubeconfig=/etc/kubernetes/scheduler.conf
    - --leader-elect=true
    image: k8s.gcr.io/kube-scheduler:v1.16.0
    imagePullPolicy: IfNotPresent
    livenessProbe:
      failureThreshold: 8
...
```

on master node we can inspect kube-apiserver process

ps -aux | grep kube-scheduler

---



## Kubelet

registers a node with k8s cluster

monitors node and pods

**kubeadm tool doesnot deploy kubelet**

download kubelet from k8s releases page, extract it , run it as a service


on a node we can inspect kube-apiserver process

ps -aux | grep kubelet

---


## Kube Proxy

in kubernetes every pod can reach every other pod

this is accomplished by deploying a pod networking solution to the cluster

pods are made available as a stable ip using services

services cannot join a pod network as it has no interfaces or listenning processes

**Kube Proxy** monitors services and when a new service is created, it adds rules so that pods can communicate with eachother. one way it uses is iptable rules.


kube-proxy binary is available in k8s release page

kubeadm runs kube-proxy as a as a daemon-set

---
